# NZ Sensor Data

### Objectives 
1. Retrieve all NZ sensor data for a sliding temporal window and categorize them by types and location.

### DEFINE data services and software modules

International Federation Data of Seismic Networks (FDSN) is a global standard and a [data service](http://www.fdsn.org/services/). Therefore, acquiring the data in FSDN also allows reusing available ObsPy libraries. Inherit the classes
1. ObsPy
   1. FSDN as Client data sources
   1. core to read data and utilize datatime features
   

In [17]:
import glob
from obspy.clients.fdsn import Client
from obspy.core import read, UTCDateTime
from datetime import date

# Set FDSN client URLs
#arc_client  = 'GEONET'
arc_client  = 'http://service.geonet.org.nz'
nrt_client = 'http://service-nrt.geonet.org.nz'
print("Available clients: \n*archive client:",arc_client,"\n*near real-time client:",nrt_client)

# Establish start and end time for retrieving waveform data
t_start = UTCDateTime.now()-518400 #6 days ago = 60s x 60m x 24h x 6d
t_end = t_start+300
print('Waveform startime: ', t_start, '\n & ending time: ', t_end)

days7 = UTCDateTime.now()-604800 #7 days ago
days8 = UTCDateTime.now()-691200 #8 days ago

# Select the appropriate client based on start and end times
if t_end < days7:
    client = Client(arc_client)
    print("Appropriate client is archive client", arc_client)
elif t_start > days8:
    client = Client(nrt_client)
    print("Appropriate client is near real-time client", nrt_client)
else:
    print("Time range requires both clients")

Available clients: 
*archive client: http://service.geonet.org.nz 
*near real-time client: http://service-nrt.geonet.org.nz
Waveform startime:  2020-10-04T08:41:11.305245Z 
 & ending time:  2020-10-04T08:46:11.305245Z
Appropriate client is near real-time client http://service-nrt.geonet.org.nz


### Try connecting to the client data services

We first try connecting to the data services to retrieve data from a single known sensor. It will allow to confirm that the services are working.

The [GeoNet FDSN webservices](https://www.geonet.org.nz/data/tools/FDSN), in the interest of getting the most complete waveform data offers two service:
1. __archive__ service, service.geonet.org.nz - (or the "GEONET" shortcut if using obspy) - __complete__ set of waveform data that is 7 days behind the present time
1. __near real-time__ service, service-nrt.geonet.org.nz - possible __incomplete__ set but of the most recent waveform data for the last 8 days (for anything older use the archive service above).

In [16]:
#first try to get data from archive server
try:
    print('trying client:',client,'\n')
    st = client.get_waveforms('NZ', 'KRVZ', '10', 'EHZ', t_start, t_end)
    print('successful')
    print(st)
#if this raises an exception, try the near real-time server
except:
    print('unable to retrieve data from client')

trying client: FDSN Webservice Client (base url: http://service-nrt.geonet.org.nz)
Available Services: 'dataselect' (v1.1), 'event' (v1.1), 'station' (v1.1), 'available_event_catalogs', 'available_event_contributors'

Use e.g. client.help('dataselect') for the
parameter description of the individual services
or client.help() for parameter description of
all webservices. 

successful
1 Trace(s) in Stream:
NZ.KRVZ.10.EHZ | 2020-10-04T08:33:31.313125Z - 2020-10-04T08:38:30.223125Z | 100.0 Hz, 29892 samples


### Get all Station details

Location codes reserved for the seismic sensors are:
* 1? - weak motion sensors
* 2? - strong motion sensors

Channel codes 
- First letter of the code represents a combination of sampling rate and sensor bandwidth
* U (Ultra Long Period sampled at 0.01Hz, or SOH sampled at 0.01Hz)
* V (Very Long Period sampled at 0.1Hz, or SOH sampled at 0.1Hz)
* L (Broad band sampled at 1Hz, or SOH sampled at 1Hz)
* B (Broad band sampled at between 10 and 80 Hz, usually 10 or 50 Hz)
* S (Short-period sampled at between 10 and 80 Hz, usually 50 Hz)
* H (High Broad band sampled at or above 80Hz, generally 100 or 200 Hz)
* E (Extremely Short-period sampled at or above 80Hz, generally 100 Hz)

- The second letter represents the sensor type, e.g.(listed are the ones relevant to seismometers
* H (Weak motion sensor, e.g. measuring velocity)
* N (Strong motion sensor, e.g. measuring acceleration)
* L (Low gain sensor, usually velocity)
* M (Mass position, used for monitoring broadband sensors)

### Get waveform for all seismographs
Location codes reserved for the seismic sensors are:
* 1? - weak motion sensors
* 2? - strong motion sensors

Since waveform retrieval does not allow for using multiple wildcards for location, we have to iterate for each location and then merge them into a single Stream. 

In [68]:
t_start = UTCDateTime.now()-518400 #6 days ago
t_end = t_start+300
print('Waveform startime: ', t_start, '\n & ending time: ', t_end)
print(client)

# st_waveforms = client.get_waveforms(network="NZ", location="1?,2?", station="*", 
#                       channel="U*,V*,L*,B*,S*,H*,E*", starttime=t_start, endtime = t_end)

#get weak motion waveforms with location id strating with 1
try:
    wmst_waveforms = client.get_waveforms(network='NZ', station='*', location='1?', channel='*', 
                                          starttime=t_start, endtime=t_end)
    print(wmst_waveforms)
except Exception as err:
    print("Exception message:", err)

try:
    smst_waveforms = client.get_waveforms(network='NZ', station='*', location='2?', channel='*',
                                          starttime=t_start, endtime=t_end)
    print(smst_waveforms)
except Exception as err:
    print("Exception message:", err)        
    
#    st_waveforms.plot()

Waveform startime:  2020-10-04T09:57:23.854705Z 
 & ending time:  2020-10-04T10:02:23.854705Z
FDSN Webservice Client (base url: http://service-nrt.geonet.org.nz)
Available Services: 'dataselect' (v1.1), 'event' (v1.1), 'station' (v1.1), 'available_event_catalogs', 'available_event_contributors'

Use e.g. client.help('dataselect') for the
parameter description of the individual services
or client.help() for parameter description of
all webservices.
835 Trace(s) in Stream:

NZ.ABAZ.10.EHE | 2020-10-04T09:57:28.617936Z - 2020-10-04T10:02:23.857936Z | 100.0 Hz, 29525 samples
...
(833 other traces)
...
NZ.WVZ.10.LHZ | 2020-10-04T09:59:54.071862Z - 2020-10-04T10:02:24.071862Z | 1.0 Hz, 151 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]
Exception message: Unknown Error (timeout): timed out


In [69]:
print(st_waveforms.__str__(extended=True))

388 Trace(s) in Stream:
NZ.ABAZ.10.EHE | 2020-10-04T09:43:54.558393Z - 2020-10-04T09:48:49.098393Z | 100.0 Hz, 29455 samples
NZ.ABAZ.10.EHN | 2020-10-04T09:43:53.778393Z - 2020-10-04T09:48:49.098393Z | 100.0 Hz, 29533 samples
NZ.ABAZ.10.EHZ | 2020-10-04T09:43:49.178393Z - 2020-10-04T09:48:49.098393Z | 100.0 Hz, 29993 samples
NZ.AKCZ.10.EHE | 2020-10-04T09:43:54.828400Z - 2020-10-04T09:48:49.098400Z | 100.0 Hz, 29428 samples
NZ.AKCZ.10.EHN | 2020-10-04T09:43:52.648400Z - 2020-10-04T09:48:49.098400Z | 100.0 Hz, 29646 samples
NZ.AKCZ.10.EHZ | 2020-10-04T09:43:52.568400Z - 2020-10-04T09:48:49.098400Z | 100.0 Hz, 29654 samples
NZ.ALRZ.10.EHE | 2020-10-04T09:43:50.070002Z - 2020-10-04T09:48:49.100002Z | 100.0 Hz, 29904 samples
NZ.ALRZ.10.EHN | 2020-10-04T09:43:50.300002Z - 2020-10-04T09:48:49.100002Z | 100.0 Hz, 29881 samples
NZ.ALRZ.10.EHZ | 2020-10-04T09:43:50.240002Z - 2020-10-04T09:48:49.100002Z | 100.0 Hz, 29887 samples
NZ.AMCZ.10.EHE | 2020-10-04T09:43:50.038393Z - 2020-10-04T09:48:49.

### Resources
1. GEONET [Stream Naming Conventions](https://www.geonet.org.nz/data/supplementary/channels) are based on historical usage together with recommendations from the [SEED manual](https://www.fdsn.org/seed_manual/SEEDManual_V2.4.pdf)
